In [43]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
#!pip install emoji
import emoji as emoji
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Sequential


In [45]:
def run():
  train_X = pd.read_csv('emojify_train_x.csv',header=None)
  test_X = pd.read_csv('emojiy_test_x.csv',header=None)

  train_Y = pd.read_csv('Emojify_Y_train.csv',header=None)

  test_Y = pd.read_csv('emojiy_y_test.csv',header=None)

  frames = [train_X , test_X]
  X = pd.concat(frames)
  frames = [train_Y , test_Y]
  Y = pd.concat(frames)
  #X.shape
  #Y.shape

  import re as s
  def clean(train):
    List = train.iloc[:,0].tolist()
    Regex = str.maketrans("","","'")
    word = [s.translate(Regex) for s in List]
    tokenized_sent = []
    for s in word:
      tokenized_sent.append(word_tokenize(s.lower())) 
    return word , tokenized_sent

  trained_list , trained_and_tokened_list = clean(X)

  emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                      "1": ":baseball:",
                      "2": ":beaming_face_with_smiling_eyes:",
                      "3": ":downcast_face_with_sweat:",
                      "4": ":fork_and_knife:",
                    }


  module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
  model_e = hub.load(module_url)
  #print ("module %s loaded" % module_url)

  sentence_embeddings = model_e(trained_list)
  category = Y.iloc[:,0].tolist()
  category_ohe = pd.get_dummies(category)

  X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings.numpy(), category_ohe, test_size=0.33, random_state=42)

  model = Sequential()
  model.add(Dense(64, input_dim=512, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(5, activation='sigmoid'))
  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['acc'])
  model.fit(X_train,y_train,epochs=40,shuffle=True,validation_split=0.1)


  def getEmoji(sentence):
    query = []
    query.append(sentence)
    inp = model_e(query)
    out =  model.predict_classes(inp)
    print('Statement is :  '+query[0])
    print('Classification Result is :  '+emoji.emojize(emoji_dictionary[str(out[0])]))


In [36]:
#emoji.EMOJI_ALIAS_UNICODE
#model.evaluate(X_test,y_test)
# for e in emoji_dictionary.values():
#     print(emoji.emojize(e))

In [48]:
run()

Epoch 1/40
4/4 [==============================] - 1s 87ms/step - loss: 1.6094 - acc: 0.1772 - val_loss: 1.5646 - val_acc: 0.6154
Epoch 2/40
4/4 [==============================] - 0s 23ms/step - loss: 1.5617 - acc: 0.7001 - val_loss: 1.5098 - val_acc: 0.8462
Epoch 3/40
4/4 [==============================] - 0s 27ms/step - loss: 1.5140 - acc: 0.7859 - val_loss: 1.4398 - val_acc: 0.8462
Epoch 4/40
4/4 [==============================] - 0s 26ms/step - loss: 1.4615 - acc: 0.7421 - val_loss: 1.3555 - val_acc: 0.8462
Epoch 5/40
4/4 [==============================] - 0s 24ms/step - loss: 1.3867 - acc: 0.7993 - val_loss: 1.2513 - val_acc: 0.8462
Epoch 6/40
4/4 [==============================] - 0s 25ms/step - loss: 1.3171 - acc: 0.7653 - val_loss: 1.1320 - val_acc: 0.8462
Epoch 7/40
4/4 [==============================] - 0s 25ms/step - loss: 1.2032 - acc: 0.8000 - val_loss: 0.9986 - val_acc: 0.8462
Epoch 8/40
4/4 [==============================] - 0s 24ms/step - loss: 1.0882 - acc: 0.8368 - val

In [50]:
sentence = input()
getEmoji(sentence)

He is crying
Statement is :  He is crying
Classification Result is :  😓
